- title: The Statefulness of Systems and Software
- date: 2020-07-25
- modified: 2020-07-25 00:00
- category: Engineering
- tags: Systems Engineering, Software Engineering
- slug: StatefulSystems
- summary: Breakdown of how most systems have state
- header_cover: images/mathematics-banner.jpg

Most of the time in software engineering, designers tend to shun or gravitate towards utilizing the internal state of the module they're developing. This post isn't to discuss that. Rather, it's to show that the concept of *state* is a embedded in a wide range of structures. But first, let's try to define what's meant by state itself.

# The Stateful Nature of Things
Usually in signal processing and many other software applications, we like to think of things as a function. Some set of inputs **x** is passed to a function $f()$, and we get a response **y** from that function.

![Time Invariant System](../images/TI_System.png)
<figcaption><i>Fig. 1 - Time-Invariant System Flow</i></figcaption>

A lot of things seem to work that way: a light bulb filament gives off light proportionate to the energy applied to it. This lack of statefulness implies *time invariance*: It doesn't matter when energy is applied, the filament gives off light.

It's a convenient way to understand systems. This is how *data driven* systems are designed: Data goes in, transformed data comes out. But time invariance places specific requirements on the inputs themselves. When such a system is given an *impulse*, it changes its response only then and provides a transient response afterwards (which can include no response at all!). Take for example listening to the radio: if the signal is disrupted, but what if we don't want the music to abruptly stop? Many streaming services have *buffering* built into them to cache the stream so that disruptions in the signal are masked by the buffer itself. In a sense, this buffer cache is providing a state to the application: as long as it isn't empty, normal operations can progress!

Digital control systems are usually designed considering that the input signal may be *latent* or *dropped* an update, but they still have requirements to produce their output following some periodic. Because of this, a lot of signal processing implementations follow this model.

![Time Invariant System](../images/nonTI_System.png)
<figcaption><i>Fig. 2 - Data-Robust System Flow</i></figcaption>

The above model now has two **triggers** for generating an output: one is upon receipt of new data, and another is when the timer expires. This simple model guarantees a data rate of at-least the timer rate, which is applicable in a lot of design circumstances. But the internal **state** of the controller changes based on the trigger because the logic for when no new data is available will activate or deactivate based on the trigger.

# State Modeling
Lets take a look at some sample code describing a system like in Figure 2.

```c++
#include <MyInputApi.h>
#include <MyOutputApi.h>
#include <MyOverseerApi.h>
#include <MyTransformApi.h>

void MyMain()
{
  MyTransformApi hiddenFunc;
  try
  {
    // Initialize the interfaces
    MyInputApi inputIf;
    inputIf.connect();

    MyOutputApi outputIf;
    outputIf.connect();

    std::string buf;
    while(MyOverseerApi::getInstance().keepRunning())
    {
      buf.clear();
      MyTransformApi::OutData output;

      if(inputIf.getNext(buf))
      {
        // Got new data
        output = hiddenFunc.magic(buf);
      }
      else
      {
        // Data wasn't available
        output = hiddenFunc.extrapolateFromLast();
      }
      outputIf.send(output);
    }    
  }
  catch(...)
  {
    std::cout << "Caught an exception, so aborting!" << std::endl;
  }
}
```

In general, the above code is very functional and has a simple decision flow that can be described like this: